# VacationPy

## Import Libraries and Load the Weater and Coordinates Data

In [60]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

pd.options.plotting.backend = 'holoviews'

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [61]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,-1.20,73,100,6.17,US,1669917143
1,1,ushuaia,-54.8000,-68.3000,14.81,44,75,7.20,AR,1669917397
2,2,beidao,34.5686,105.8933,-3.55,46,51,1.92,CN,1669917397
3,3,busselton,-33.6500,115.3333,11.71,92,1,3.23,AU,1669917189
4,4,rikitea,-23.1203,-134.9692,23.43,74,80,4.74,PF,1669917398


# Step 1: Create a map that displays the humidity for each city. 

In [62]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5, 
    title = "Cities and their Humidity"   
)

# Display the map plot
map_plot_1

#How to readjust the legend? 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

# Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 35) & (city_data_df["Max Temp"] >15) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] < 100),:]

ideal_cities_df.dropna(how = "any")  #drops any rows that have null (missing) data

ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,san cristobal,7.7669,-72.2250,18.10,78,85,2.15,VE,1669917399
7,7,arroyos y esteros,-25.0667,-57.1000,31.78,43,65,1.05,PY,1669917400
18,18,puri,19.8000,85.8500,22.29,68,0,1.88,IN,1669917406
21,21,bredasdorp,-34.5322,20.0403,18.94,81,61,3.76,ZA,1669917407
22,22,bushenyi,-0.5853,30.2114,16.26,96,99,0.76,UG,1669917408
23,23,balikpapan,-1.2675,116.8289,24.55,94,20,2.06,ID,1669917408
25,25,saint-philippe,-21.3585,55.7679,23.77,78,49,3.31,RE,1669917409
33,33,kruisfontein,-34.0033,24.7314,19.38,82,26,1.40,ZA,1669917164
34,34,bage,-31.3314,-54.1069,33.88,43,75,3.09,BR,1669917413
35,35,bara,10.3744,10.7288,28.88,25,18,3.25,NG,1669917414


# Step 3: Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

In [64]:
hotel_df = ideal_cities_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,san cristobal,VE,7.7669,-72.2250,78,
7,arroyos y esteros,PY,-25.0667,-57.1000,43,
18,puri,IN,19.8000,85.8500,68,
21,bredasdorp,ZA,-34.5322,20.0403,81,
22,bushenyi,UG,-0.5853,30.2114,96,
23,balikpapan,ID,-1.2675,116.8289,94,
25,saint-philippe,RE,-21.3585,55.7679,78,
33,kruisfontein,ZA,-34.0033,24.7314,82,
34,bage,BR,-31.3314,-54.1069,43,
35,bara,NG,10.3744,10.7288,25,


# Step 4: For each city, use the Geoapify API to find the first hotel located eithin 10,000 meters of your coordinates

In [65]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 5 	 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as e: #(KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(e)
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san cristobal - nearest hotel: Terra Sur
list index out of range
arroyos y esteros - nearest hotel: No hotel found
puri - nearest hotel: Divine Beach Homestay
bredasdorp - nearest hotel: Victoria Hotel
bushenyi - nearest hotel: Flagship Hotel
balikpapan - nearest hotel: Hotel Grand Senyiur
saint-philippe - nearest hotel: Le Baril
list index out of range
kruisfontein - nearest hotel: No hotel found
'name'
bage - nearest hotel: No hotel found
list index out of range
bara - nearest hotel: No hotel found
list index out of range
tucuma - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house


,City,Country,Lat,Lng,Humidity,Hotel Name
5,san cristobal,VE,7.7669,-72.2250,78,Terra Sur
7,arroyos y esteros,PY,-25.0667,-57.1000,43,No hotel found
18,puri,IN,19.8000,85.8500,68,Divine Beach Homestay
21,bredasdorp,ZA,-34.5322,20.0403,81,Victoria Hotel
22,bushenyi,UG,-0.5853,30.2114,96,Flagship Hotel
23,balikpapan,ID,-1.2675,116.8289,94,Hotel Grand Senyiur
25,saint-philippe,RE,-21.3585,55.7679,78,Le Baril
33,kruisfontein,ZA,-34.0033,24.7314,82,No hotel found
34,bage,BR,-31.3314,-54.1069,43,No hotel found
35,bara,NG,10.3744,10.7288,25,No hotel found


In [66]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Aloe guest house',
    'street': 'Main Road',
    'suburb': 'Mount Pleasant',
    'town': 'Hermanus',
    'city': 'Overstrand Local Municipality',
    'county': 'Overberg District Municipality',
    'state': 'Western Cape',
    'postcode': '7200',
    'country': 'South Africa',
    'country_code': 'za',
    'lon': 19.2345494,
    'lat': -34.4192217,
    'formatted': 'Aloe guest house, Main Road, Mount Pleasant, Overstrand Local Municipality, 7200, South Africa',
    'address_line1': 'Aloe guest house',
    'address_line2': 'Main Road, Mount Pleasant, Overstrand Local Municipality, 7200, South Africa',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'A

# Step 5: Add the hotel name and the country to hover message

In [68]:
%%capture --no-display

# Configure the map plot
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["City", "Lat", "Lng", "Humidity", "Hotel Name", "Country"]    
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)